# Homework 3
W261
Chris Dailey

christopher.dailey@gmail.com

## HW 3.1
Use Hadoop Counters to identify the number of complaints pertaining to debt collection, mortgage and other categories

In [4]:
!head Consumer_Complaints.csv -n 30

Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?
1114245,Debt collection,Medical,Disclosure verification of debt,Not given enough info to verify debt,FL,32219,Web,11/13/2014,11/13/2014,"Choice Recovery, Inc.",Closed with explanation,Yes,
1114488,Debt collection,Medical,Disclosure verification of debt,Right to dispute notice not received,TX,75006,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,
1114255,Bank account or service,Checking account,Deposits and withdrawals,,NY,11102,Web,11/13/2014,11/13/2014,"FNIS (Fidelity National Information Services, Inc.)",In progress,Yes,
1115106,Debt collection,"Other (phone, health club, etc.)",Communication tactics,Frequent or repeated calls,GA,31721,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,
1115890,Credit reporting,,Incorrect information on credit report,Information is

In [8]:
!hdfs dfs -mkdir /user/ubuntu/HW3
!hdfs dfs -mkdir /user/ubuntu/HW3/3.1

In [6]:
!hdfs dfs -put Consumer_Complaints.csv /user/ubuntu/HW3/

In [8]:
%%writefile mapper.py
#!/usr/bin/python

import sys, re

for line in sys.stdin:
    splits = re.split(",", line)
    if splits[1].lower() == "debt collection":
        sys.stderr.write("reporter:counter:Type,DebtCollection,1\n")
    elif splits[1].lower() == "mortgage":
        sys.stderr.write("reporter:counter:Type,Mortgage,1\n")
    else:
        sys.stderr.write("reporter:counter:Type,Other,1\n")

Overwriting mapper.py


In [9]:
!chmod +x mapper.py

In [10]:
!hdfs dfs -rm /user/ubuntu/HW3/3.1/output/*
!hdfs dfs -rmdir /user/ubuntu/HW3/3.1/output
!hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-mapper /home/ubuntu/repos/261/HW3/mapper.py \
-input /user/ubuntu/HW3/Consumer_Complaints.csv \
-output /user/ubuntu/HW3/3.1/output

16/01/30 02:00:32 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/HW3/3.1/output/_SUCCESS
16/01/30 02:00:32 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/HW3/3.1/output/part-00000
packageJobJar: [/tmp/hadoop-unjar2402735397871742075/] [] /tmp/streamjob5694268014097702284.jar tmpDir=null
16/01/30 02:00:36 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 02:00:36 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 02:00:37 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/30 02:00:37 INFO mapreduce.JobSubmitter: number of splits:2
16/01/30 02:00:37 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453815382601_0023
16/01/30 02:00:37 INFO impl.YarnClientImpl: Submitted application application_1453815382601_0023
16/01/30 02:00

## HW 3.2
Perform a word count analysis of the phrase "foo foo quux labs foo bar quux" and count the number of mappers and reducers and explain the results.

In [50]:
%%writefile mapper.py
#!/usr/bin/python

import sys, re

sys.stderr.write("reporter:counter:TaskCounter,Mapper,1\n")

for line in sys.stdin:
    for word in [x.strip().lower() for x in re.split(" ", line)]:
        print(word + ", " + str(1))

Overwriting mapper.py


In [24]:
!echo "test test test test" | python mapper.py

reporter:counter:MapperCount,1
test, 1
test, 1
test, 1
test, 1


In [49]:
%%writefile reducer.py
#!/usr/bin/python

import sys, re

sys.stderr.write("reporter:counter:TaskCounter,Reducer,1\n")

current_word = ""
current_sum = 0
for line in sys.stdin:
    splits = [x.lower().strip() for x in re.split(",", line)]
    if (len(splits)) < 2:
        continue
    if current_word == "":
        current_word = splits[0]
    if splits[0] != current_word:
        print(current_word + " " + str(current_sum))
        current_word = splits[0]
        current_sum = int(splits[1])
    else:
        current_sum += int(splits[1])
print(current_word + " " + str(current_sum))

Overwriting reducer.py


In [39]:
!chmod +x reducer.py

In [40]:
!echo "foo foo quux labs foo bar quux" | python mapper.py | sort | python reducer.py

reporter:counter:ReducerCount,1
reporter:counter:MapperCount,1
bar 1
foo 3
labs 1
quux 2


In [41]:
!hdfs dfs -mkdir /user/ubuntu/HW3/3.2

In [46]:
!echo "foo foo quux labs foo bar quux" > oneline.txt ; cat oneline.txt

foo foo quux labs foo bar quux


In [47]:
!hdfs dfs -put oneline.txt /user/ubuntu/HW3/3.2

In [52]:
!hdfs dfs -rm /user/ubuntu/HW3/3.2/output/*
!hdfs dfs -rmdir /user/ubuntu/HW3/3.2/output

!hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-mapper /home/ubuntu/repos/261/HW3/mapper.py \
-reducer ~/repos/261/HW3/reducer.py \
-input /user/ubuntu/HW3/3.2/oneline.txt \
-output /user/ubuntu/HW3/3.2/output

16/01/29 00:50:03 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/HW3/3.2/output/_SUCCESS
16/01/29 00:50:03 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/HW3/3.2/output/part-00000
packageJobJar: [/tmp/hadoop-unjar519971968270769378/] [] /tmp/streamjob435509402778434573.jar tmpDir=null
16/01/29 00:50:08 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/29 00:50:08 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/29 00:50:08 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/29 00:50:08 INFO mapreduce.JobSubmitter: number of splits:2
16/01/29 00:50:08 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453815382601_0021
16/01/29 00:50:09 INFO impl.YarnClientImpl: Submitted application application_1453815382601_0021
16/01/29 00:50:0

Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper and Reducer based WordCount (i.e., no combiners used anywhere)  using user defined Counters to count up how many time the mapper and reducer are called

In [11]:
%%writefile mapper.py
#!/usr/bin/python

import sys, re

WORD_RE = re.compile(r"[\w']+")

sys.stderr.write("reporter:counter:TaskCounter,Mapper,1\n")

for line in sys.stdin:
    splits = [x.lower().strip() for x in re.split(",", line)]
    issues = splits[3]
    for word in WORD_RE.findall(issues):
        print(word+", 1")

Overwriting mapper.py


In [3]:
!echo ["","","","test this is a test"] | python mapper.py

reporter:counter:TaskCounter,Mapper,1
test, 1
this, 1
is, 1
a, 1
test, 1


In [12]:
%%writefile reducer.py
#!/usr/bin/python

import sys, re

sys.stderr.write("reporter:counter:TaskCounter,Reducer,1\n")

current_word = ""
current_sum = 0
for line in sys.stdin:
    splits = [x.lower().strip() for x in re.split(",", line)]
    if (len(splits)) < 2:
        continue
    if current_word == "":
        current_word = splits[0]
    if splits[0] != current_word:
        print(current_word + " " + str(current_sum))
        current_word = splits[0]
        current_sum = int(splits[1])
    else:
        current_sum += int(splits[1])
print(current_word + " " + str(current_sum))

Overwriting reducer.py


In [13]:
!echo ["","","","test this is a test"] | python mapper.py | sort | python reducer.py

reporter:counter:TaskCounter,Mapper,1
reporter:counter:TaskCounter,Reducer,1
a 1
is 1
test 2
this 1


In [14]:
!hdfs dfs -rm /user/ubuntu/HW3/3.2/output/*
!hdfs dfs -rmdir /user/ubuntu/HW3/3.2/output

!hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-mapper /home/ubuntu/repos/261/HW3/mapper.py \
-reducer ~/repos/261/HW3/reducer.py \
-input /user/ubuntu/HW3/Consumer_Complaints.csv \
-output /user/ubuntu/HW3/3.2/output

16/01/30 02:02:13 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/HW3/3.2/output/_SUCCESS
16/01/30 02:02:13 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/HW3/3.2/output/part-00000
packageJobJar: [/tmp/hadoop-unjar5712215938276641105/] [] /tmp/streamjob6344048016643955809.jar tmpDir=null
16/01/30 02:02:17 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 02:02:18 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 02:02:18 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/30 02:02:18 INFO mapreduce.JobSubmitter: number of splits:2
16/01/30 02:02:18 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453815382601_0024
16/01/30 02:02:18 INFO impl.YarnClientImpl: Submitted application application_1453815382601_0024
16/01/30 02:02

The TaskCounter reads:<br>
		Mapper=2<br>
		Reducer=1
    

In [15]:
!hdfs dfs -cat /user/ubuntu/HW3/3.2/output/part-00000

a 3503	
account 20681	
acct 163	
action 2505	
advance 240	
advertising 1193	
amount 98	
amt 71	
an 2505	
and 16448	
application 8868	
applied 139	
apply 118	
apr 3431	
arbitration 168	
are 3821	
atm 2422	
attempts 11848	
available 274	
balance 597	
bank 202	
bankruptcy 222	
being 5663	
billing 8158	
by 5663	
can't 1999	
cancelling 2795	
card 4405	
cash 240	
caused 5663	
changes 350	
charged 976	
charges 131	
checks 75	
closing 2795	
club 12545	
collect 11848	
collection 1907	
communication 6920	
company's 4858	
cont'd 11848	
contact 3053	
convenience 75	
costs 4350	
credit 55251	
credited 92	
customer 2734	
day 71	
dealing 1944	
debit 2422	
debt 19309	
decision 2774	
decrease 1149	
delay 243	
delinquent 1061	
deposits 10555	
determination 1490	
did 139	
didn't 925	
disclosure 5214	
disclosures 64	
dispute 904	
disputes 6938	
embezzlement 3276	
expect 807	
false 2508	
fee 3198	
fees 807	
for 929	
forbearance 350	
fraud 3842	
funds 5663	
get 4357	
getting 291	
health 12545	
i 925	
identi

Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper, Reducer, and standalone combiner (i.e., not an in-memory combiner) based WordCount using user defined Counters to count up how many time the mapper, combiner, reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. 

In [16]:
%%writefile combiner.py
#!/usr/bin/python

import sys, re

#this is the exact same as the reducer except the output is modified
#to include a comma between the key and value to fulfill the contract 
#that the combiner accept and emit keys in the same format as the mapper

sys.stderr.write("reporter:counter:TaskCounter,Combiner,1\n")

current_word = ""
current_sum = 0
for line in sys.stdin:
    splits = [x.lower().strip() for x in re.split(",", line)]
    if (len(splits)) < 2:
        continue
    if current_word == "":
        current_word = splits[0]
    if splits[0] != current_word:
        print(current_word + ", " + str(current_sum))
        current_word = splits[0]
        current_sum = int(splits[1])
    else:
        current_sum += int(splits[1])
print(current_word + ", " + str(current_sum))

Writing combiner.py


In [17]:
!chmod +x combiner.py

In [18]:
!hdfs dfs -rm /user/ubuntu/HW3/3.2/output/*
!hdfs dfs -rmdir /user/ubuntu/HW3/3.2/output

!hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-mapper /home/ubuntu/repos/261/HW3/mapper.py \
-reducer ~/repos/261/HW3/reducer.py \
-combiner /home/ubuntu/repos/261/HW3/combiner.py \
-input /user/ubuntu/HW3/Consumer_Complaints.csv \
-output /user/ubuntu/HW3/3.2/output

16/01/30 02:54:25 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/HW3/3.2/output/_SUCCESS
16/01/30 02:54:26 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/HW3/3.2/output/part-00000
packageJobJar: [/tmp/hadoop-unjar1646522515064830425/] [] /tmp/streamjob1924805495545833480.jar tmpDir=null
16/01/30 02:54:30 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 02:54:30 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 02:54:30 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/30 02:54:30 INFO mapreduce.JobSubmitter: number of splits:2
16/01/30 02:54:30 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453815382601_0025
16/01/30 02:54:30 INFO impl.YarnClientImpl: Submitted application application_1453815382601_0025
16/01/30 02:54

Using a single reducer: What are the top 50 most frequent terms in your word count analysis? Present the top 50 terms and their frequency and their relative frequency. If there are ties please sort the tokens in alphanumeric/string order. Present bottom 10 tokens (least frequent items).

In [19]:
#Build an order-inverting mapper

In [64]:
%%writefile mapper.py
#!/usr/bin/python

import sys, re

WORD_RE = re.compile(r"[\w']+")

sys.stderr.write("reporter:counter:TaskCounter,Mapper,1\n")

marginal = 0

for line in sys.stdin:
    splits = [x.lower().strip() for x in re.split(",", line)]
    issues = splits[3]
    for word in WORD_RE.findall(issues):
        print(word+"\t1")
        marginal += 1
print("***\t" + str(marginal))

Overwriting mapper.py


In [65]:
#Build an inversion-aware reducer

In [71]:
%%writefile reducer.py
#!/usr/bin/python

import sys, re

sys.stderr.write("reporter:counter:TaskCounter,Reducer,1\n")

marginal = 0

current_word = "***"
current_sum = 0
for line in sys.stdin:  
    splits = [x.lower().strip() for x in re.split("\t", line)]
    if (len(splits)) < 2:
        continue
    if splits[0] != current_word:
        if current_word != "***":
            print(current_word + "\t" + str(current_sum) + "\t" + str(1.*current_sum/marginal))
        current_word = splits[0]
        current_sum = int(splits[1])
    else:
        if current_word == "***":
            marginal += int(splits[1])
        else:
            current_sum += int(splits[1])
print(current_word + "\t" + str(current_sum) + "\t" + str(1.*current_sum/marginal))

Overwriting reducer.py


In [72]:
!cat Consumer_Complaints_mini.csv | python mapper.py | sort | python reducer.py

reporter:counter:TaskCounter,Mapper,1
reporter:counter:TaskCounter,Reducer,1
a	2	0.0009474182851729038
account	19	0.009000473709142587
action	3	0.0014211274277593558
advertising	1	0.0004737091425864519
an	3	0.0014211274277593558
and	18	0.008526764566556135
application	3	0.0014211274277593558
applied	1	0.0004737091425864519
apr	3	0.0014211274277593558
are	3	0.0014211274277593558
atm	2	0.0009474182851729038
attempts	37	0.01752723827569872
available	1	0.0004737091425864519
balance	1	0.0004737091425864519
being	9	0.004263382283278067
billing	1	0.0004737091425864519
by	9	0.004263382283278067
cancelling	1	0.0004737091425864519
can't	3	0.0014211274277593558
card	2	0.0009474182851729038
caused	9	0.004263382283278067
charged	2	0.0009474182851729038
closing	1	0.0004737091425864519
club	55	0.026054002842254856
collect	37	0.01752723827569872
communication	24	0.011369019422074847
company's	40	0.018948365703458078
contact	9	0.004263382283278067
cont'd	37	0.01752723827569872
costs	4	0.001894836570345

In [73]:
!hdfs dfs -rm /user/ubuntu/HW3/3.2/output/*
!hdfs dfs -rmdir /user/ubuntu/HW3/3.2/output

!hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-mapper /home/ubuntu/repos/261/HW3/mapper.py \
-reducer ~/repos/261/HW3/reducer.py \
-input /user/ubuntu/HW3/Consumer_Complaints.csv \
-output /user/ubuntu/HW3/3.2/output

16/01/31 16:05:23 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/HW3/3.2/output/_SUCCESS
16/01/31 16:05:23 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/HW3/3.2/output/part-00000
packageJobJar: [/tmp/hadoop-unjar7543454808455630869/] [] /tmp/streamjob4492476671369596750.jar tmpDir=null
16/01/31 16:05:27 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/31 16:05:27 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/31 16:05:27 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/31 16:05:27 INFO mapreduce.JobSubmitter: number of splits:2
16/01/31 16:05:27 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453815382601_0051
16/01/31 16:05:28 INFO impl.YarnClientImpl: Submitted application application_1453815382601_0051
16/01/31 16:05

In [74]:
!hdfs dfs -cat /user/ubuntu/HW3/3.2/output/part-00000

a	3503	0.00357272895094
account	20681	0.0210926655536
acct	163	0.000166244595776
action	2505	0.00255486326637
advance	240	0.000244777318934
advertising	1193	0.0012167472562
amount	98	9.99507385646e-05
amt	71	7.24132901845e-05
an	2505	0.00255486326637
and	16448	0.0167754055909
application	8868	0.0090445219346
applied	139	0.000141766863882
apply	118	0.000120348848476
apr	3431	0.00349929575526
arbitration	168	0.000171344123254
are	3821	0.00389705889852
atm	2422	0.00247021111024
attempts	11848	0.0120838403114
available	274	0.000279454105783
balance	597	0.000608883580847
bank	202	0.000206020910102
bankruptcy	222	0.000226419020014
being	5663	0.00577572482134
billing	8158	0.00832038903275
by	5663	0.00577572482134
can't	1999	0.00203879108562
cancelling	2795	0.00285063586008
card	4405	0.00449268370793
cash	240	0.000244777318934
caused	5663	0.00577572482134
changes	350	0.000356966923445
charged	976	0.000995427763663
charges	131	0.000133607619918
checks	75	7.64929121668e-05
closing	2795	0.0028506

In [75]:
%%writefile mapper.py
#!/usr/bin/python

import sys

for line in sys.stdin:
    print(line.strip())

Overwriting mapper.py


In [76]:
%%writefile reducer.py
#!/usr/bin/python

import sys,re

#ount = 0

for line in sys.stdin:
    print(line.strip())
    #ount += 1
    #f count >= 50:
    #   break

Overwriting reducer.py


In [84]:
!hdfs dfs -rm /user/ubuntu/HW3/3.2.4/output/*
!hdfs dfs -rmdir /user/ubuntu/HW3/3.2.4/output

!hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-D stream.num.map.output.key.field=3 \
-D mapreduce.partition.keypartitioner.options=-k1 \
-D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D mapreduce.partition.keycomparator.options=-k2nr \
-mapper /home/ubuntu/repos/261/HW3/mapper.py \
-reducer /home/ubuntu/repos/261/HW3/reducer.py \
-input /user/ubuntu/HW3/3.2/output/part-00000 \
-output /user/ubuntu/HW3/3.2.4/output

16/01/31 16:13:31 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/HW3/3.2.4/output/_SUCCESS
16/01/31 16:13:31 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/HW3/3.2.4/output/part-00000
packageJobJar: [/tmp/hadoop-unjar1089046432507248508/] [] /tmp/streamjob5782657830717292465.jar tmpDir=null
16/01/31 16:13:35 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/31 16:13:35 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/31 16:13:36 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/31 16:13:36 INFO mapreduce.JobSubmitter: number of splits:2
16/01/31 16:13:36 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453815382601_0055
16/01/31 16:13:36 INFO impl.YarnClientImpl: Submitted application application_1453815382601_0055
16/01/31 1

In [85]:
!hdfs dfs -cat /user/ubuntu/HW3/3.2.4/output/part-00000

payments	3226	0.00329021512867
payment	92	9.38313055912e-05
pay	3821	0.00389705889852
owed	11848	0.0120838403114
overlimit	127	0.000129527997936
out	1242	0.00126672262548
other	7886	0.00804297473796
plans	350	0.000356966923445
payoff	1155	0.00117799084737
opening	16205	0.0165275685555
on	29069	0.0296476328503
rate	3431	0.00349929575526
protection	4139	0.00422138884611
promised	274	0.000279454105783
processing	243	0.00024783703542
process	5505	0.00561457975304
problems	9484	0.00967278371986
privacy	240	0.000244777318934
practices	1003	0.00102296521204
or	22533	0.0229815305314
received	216	0.00022029958704
loan	119630	0.122011294433
line	1732	0.0017664763183
lender	2165	0.00220809539788
lease	6337	0.00646314112534
late	1797	0.00183277017552
issues	538	0.00054870915661
issue	1099	0.00112087613962
making	3226	0.00329021512867
low	5663	0.00577572482134
receive	139	0.000141766863882
of	10885	0.0111016713191
not	12353	0.0125988925866
my	10731	0.0109446058728
monitoring	1453	0.00148192268504
m

## HW 3.3
Using a single reducer: Report your findings such as number of unique products; largest basket; report the top 50 most frequently purchased items,  their frequency,  and their relative frequency (break ties by sorting the products alphabetical order) etc. using Hadoop Map-Reduce.

In [86]:
!hdfs dfs -mkdir /user/ubuntu/HW3/3.3

In [87]:
!hdfs dfs -put ./ProductPurchaseData.txt /user/ubuntu/HW3/3.3/

### Calculate number of unique products

In [98]:
%%writefile mapper.py
#!/usr/bin/python

import sys, re

products = set()

for line in sys.stdin:
    for product in [x.strip() for x in re.split(" ", line)]:
        if product not in products:
            products.add(product)
            #send only one k,v from a mapper.  The reducer is guaranteed to get
            #all the products with the same key so even if there are multiple mappers
            #the reducers will be able to deduplicate the keys they get
            print(product + "\t1")

Overwriting mapper.py


In [94]:
!head ProductPurchaseData.txt -n50 >> ProductPurchaseDataMini.txt

In [99]:
!cat ProductPurchaseDataMini.txt | python mapper.py

FRO11987	1
ELE17451	1
ELE89019	1
SNA90258	1
GRO99222	1
	1
GRO12298	1
FRO12685	1
ELE91550	1
SNA11465	1
ELE26917	1
ELE52966	1
FRO90334	1
SNA30755	1
FRO84225	1
SNA80192	1
GRO73461	1
DAI22896	1
SNA99873	1
FRO86643	1
ELE37798	1
GRO56989	1
ELE23393	1
SNA69641	1
FRO78087	1
GRO39357	1
ELE28573	1
ELE11375	1
DAI54444	1
FRO18919	1
DAI50921	1
GRO75578	1
ELE59935	1
SNA85662	1
SNA91554	1
DAI22177	1
DAI14125	1
DAI46755	1
FRO81176	1
ELE66810	1
DAI49199	1
DAI91535	1
GRO94758	1
ELE94711	1
FRO76833	1
DAI54690	1
SNA47306	1
GRO36567	1
ELE82555	1
SNA17715	1
SNA94781	1
DAI87514	1
GRO48282	1
GRO12935	1
SNA55952	1
DAI93692	1
DAI92253	1
FRO82427	1
DAI48891	1
ELE27376	1
FRO47475	1
DAI17810	1
FRO17376	1
DAI38969	1
DAI50913	1
ELE11111	1
SNA66979	1
FRO92261	1
FRO76487	1
GRO39369	1
GRO22575	1
GRO48971	1
FRO32293	1
FRO98729	1
DAI95741	1
DAI44355	1
FRO75418	1
FRO70974	1
GRO49037	1
GRO30386	1
FRO41069	1
ELE96863	1
FRO16142	1
ELE52446	1
DAI88808	1
FRO36081	1
DAI35347	1
SNA63157	1
ELE73246	1
ELE66067	1
DAI22534	1
SNA9386

In [100]:
%%writefile reducer.py
#!/usr/bin/python

import sys, re


#Start counting at one.  This will produce an inaccurate reading of 1 product if there
#are actually 0, but I don't think we need to worry about that case
product_count = 1

current_product = None

for line in sys.stdin:
    splits = re.split("\t", line)
    if current_product is None:
        current_product = splits[0]
    if splits[0] != current_product:
        product_count += 1
        
print(str(product_count))

Overwriting reducer.py


In [101]:
!cat ProductPurchaseDataMini.txt | python mapper.py | sort | python reducer.py

219


In [102]:
!hdfs dfs -rm /user/ubuntu/HW3/3.3/output/*
!hdfs dfs -rmdir /user/ubuntu/HW3/3.3/output

!hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-mapper /home/ubuntu/repos/261/HW3/mapper.py \
-reducer /home/ubuntu/repos/261/HW3/reducer.py \
-input /user/ubuntu/HW3/3.3/ProductPurchaseData.txt \
-output /user/ubuntu/HW3/3.3/output

rm: `/user/ubuntu/HW3/3.3/output/*': No such file or directory
packageJobJar: [/tmp/hadoop-unjar1043300641705263494/] [] /tmp/streamjob6934915716189978039.jar tmpDir=null
16/01/31 18:22:20 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/31 18:22:20 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/31 18:22:20 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/31 18:22:20 INFO mapreduce.JobSubmitter: number of splits:2
16/01/31 18:22:21 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453815382601_0057
16/01/31 18:22:21 INFO impl.YarnClientImpl: Submitted application application_1453815382601_0057
16/01/31 18:22:21 INFO mapreduce.Job: The url to track the job: http://ip-172-31-54-132:8088/proxy/application_1453815382601_0057/
16/01/31 18:22:21 INFO mapreduce.Job: Running job: job_1453815382601_0057
16/01/31 18:22:26 INFO mapreduce.Job: Job job_1453815382601_0057 running in uber mode : false
16/01/31 18:22:26 

In [104]:
!hdfs dfs -cat /user/ubuntu/HW3/3.3/output/part-00000

17746	


### Find the largest basket

In [105]:
%%writefile mapper.py
#!/usr/bin/python

import sys, re

#a simple in-memory combiner
largest_basket_so_far = 0

for line in sys.stdin:
    if len(re.split(" ", line)) > largest_basket_so_far:
        largest_basket_so_far = len(re.split(" ", line))
print(str(largest_basket_so_far))

Overwriting mapper.py


In [106]:
!cat ProductPurchaseDataMini.txt | python mapper.py

6
13
6
7
10
9
9
9
15
12
19
17
5
15
10
14
9
11
5
20
19
12
5
16
14
24
9
20
17
17
14
6
10
12
10
18
9
14
13
19
12
11
8
10
11
10
13
13
11
23


In [113]:
%%writefile reducer.py
#!/usr/bin/python

import sys, re

for line in sys.stdin:
    print(line.strip())
    break

Overwriting reducer.py


In [114]:
!cat ProductPurchaseDataMini.txt | python mapper.py | sort -k1nr | python reducer.py

24


In [117]:
!hdfs dfs -rm /user/ubuntu/HW3/3.3/output/*
!hdfs dfs -rmdir /user/ubuntu/HW3/3.3/output

!hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D mapreduce.partition.keycomparator.options=-k1nr \
-mapper /home/ubuntu/repos/261/HW3/mapper.py \
-reducer /home/ubuntu/repos/261/HW3/reducer.py \
-input /user/ubuntu/HW3/3.3/ProductPurchaseData.txt \
-output /user/ubuntu/HW3/3.3/output

rm: `/user/ubuntu/HW3/3.3/output/*': No such file or directory
packageJobJar: [/tmp/hadoop-unjar7891215267387553793/] [] /tmp/streamjob2479595477839468457.jar tmpDir=null
16/01/31 18:48:19 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/31 18:48:19 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/31 18:48:19 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/31 18:48:19 INFO mapreduce.JobSubmitter: number of splits:2
16/01/31 18:48:19 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453815382601_0059
16/01/31 18:48:20 INFO impl.YarnClientImpl: Submitted application application_1453815382601_0059
16/01/31 18:48:20 INFO mapreduce.Job: The url to track the job: http://ip-172-31-54-132:8088/proxy/application_1453815382601_0059/
16/01/31 18:48:20 INFO mapreduce.Job: Running job: job_1453815382601_0059
16/01/31 18:48:25 INFO mapreduce.Job: Job job_1453815382601_0059 running in uber mode : false
16/01/31 18:48:25 

In [118]:
!hdfs dfs -cat /user/ubuntu/HW3/3.3/output/part-00000

38	


### Top 50 most frequently purchased items
Their frequency,  and their relative frequency (break ties by sorting the products alphabetical order)

In [ ]:
%%writefile mapper.py
#!/bin/usr/python

import sys, re

for line in sys.stdin:
    